In [2]:
from MCTS import MCTS
from sogo.SogoGame import SogoGame, display as display_board
import numpy as np
from sogo.keras.NNet import NNetWrapper as NNet

class Config(object):
    def __init__(self):    
      self.num_sampling_moves = 30
      self.max_moves = 512  # for chess and shogi, 722 for Go.
      self.numMCTSSims = 200

      # Root prior exploration noise.
      self.root_dirichlet_alpha = 0.3  # for chess, 0.03 for Go and 0.15 for shogi.
      self.root_exploration_fraction = 0.0

      # UCB formula
      self.pb_c_base = 19652
      self.pb_c_init = 1.25

      # Load model

      self.load_model = True
      self.load_folder_file = ('./temp/','best.pth.tar')

config = Config()
game = SogoGame(4)
nn = NNet(game)
nn.load_checkpoint(*(config.load_folder_file))
mcts = MCTS(game, nn, config)

def mcts_player(x, player):
    canonical_board = game.getCanonicalForm(x, player)
    pi, root = mcts.get_action_prob(canonical_board)
    return np.argmax(pi), root



In [3]:
from NeuralNet import NeuralNet
from Game import Game

class NN(NeuralNet):
  def __init__(self,game:Game):
    self.game = game
  def predict(self, board):
    return np.ones(self.game.getActionSize())/self.game.getActionSize(), 0


dummy_nn = NN(game)
dummy_mcts = MCTS(game, dummy_nn, config)

In [4]:
import time

class Timer:
    def __init__(self, msg):
        self.msg = msg
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        if self.msg:
            print(f"{self.msg} took {self.interval:0.3f} sec")

In [5]:
def setup_board(plays,do_print=True): 
    board = game.getInitBoard()
    player = 1    
    for play in plays:
        board, player = game.getNextState(board, player,play)
    if do_print:
        display_board(board)
    return board, player

In [6]:
def test_mcts(plays, expected):
    board, player = setup_board(plays)    
    with Timer() as t:
        play, root = mcts_player(board, player)    
    new_board, new_player = gane.getNextState(board, player, play)
    display_board(new_board)
    print(f"MCTS made {'correct' if play == expected else 'incorrect' } play in  {t.interval:0.3f} sec")
    return root

In [7]:
def nn_pred(plays):
    b,p = setup_board(plays, True)
    b = game.getCanonicalForm(b,p)
    with Timer("NN prediction"):
        pi, v = nn.predict(b)
    print(f"Probs: {np.array2string(pi, precision=2, separator=',', suppress_small=True, max_line_width=200)} Value: {v[0]:0.2f}")

In [8]:
def mcts_pred(plays, root=None):
    b,p = setup_board(plays, False)
    with Timer("MCTS prediction"):
        pi, root = mcts.get_action_prob(b, p, root)
    print(f"Probs: {np.array2string(np.array(pi), precision=2, separator=',', suppress_small=True, max_line_width=200)}")
    return root

In [9]:
def mcts_only_pred(plays, root=None):
    b,p = setup_board(plays, False)
    with Timer("MCTS only prediction"):
        pi, root = dummy_mcts.get_action_prob(b, p, root)
    print(f"Probs: {np.array2string(np.array(pi), precision=2, separator=',', suppress_small=True, max_line_width=200)}")
    return root

In [69]:
config.numMCTSSims = 1000
play = [0,1,0,1,0]
nn_pred(play)
mcts_pred(play)
mcts_only_pred(play)


z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O X - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O X - - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.013 sec
Probs: [0.12,0.05,0.06,0.08,0.04,0.05,0.05,0.05,0.04,0.04,0.04,0.07,0.09,0.06,0.09,0.07] Value: 0.96
MCTS prediction took 15.910 sec
Probs: [0.76,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.01,0.01,0.03,0.03,0.01,0.02]
MCTS only prediction took 1.844 sec
Probs: [0.69,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02]


In [26]:
config.numMCTSSims = 100
play = [2,1,2,1,2]
nn_pred(play)
mcts_pred(play)
mcts_only_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.014 sec
Probs: [0.05,0.03,0.38,0.03,0.04,0.04,0.05,0.04,0.03,0.05,0.04,0.05,0.05,0.04,0.04,0.03] Value: 0.86
MCTS prediction took 1.189 sec
Probs: [0.02,0.02,0.44,0.01,0.01,0.02,0.17,0.11,0.05,0.02,0.03,0.02,0.02,0.02,0.02,0.02]
MCTS only prediction took 0.126 sec
Probs: [0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.07,0.07,0.07,0.07]


In [24]:
nn.load_checkpoint("temp/","best.pth.tar")

In [18]:
config.numMCTSSims = 100
play = [2,1,2,1,2]
nn_pred(play)
r1 = mcts_pred(play)
r0 = mcts_only_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.013 sec
Probs: [0.04,0.05,0.44,0.02,0.04,0.04,0.04,0.02,0.02,0.04,0.05,0.04,0.03,0.05,0.05,0.04] Value: 0.90
MCTS prediction took 1.370 sec
Probs: [0.01,0.15,0.41,0.01,0.05,0.01,0.01,0.01,0.03,0.04,0.11,0.03,0.01,0.05,0.01,0.06]
MCTS only prediction took 0.109 sec
Probs: [0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.07,0.07,0.07,0.07]


In [19]:
config.numMCTSSims = 100
play = [2,1,2,1,2,2]
nn_pred(play)
mcts_pred(play,r1.children[2])
mcts_only_pred(play,r0.children[2])

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - X - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.024 sec
Probs: [0.06,0.06,0.  ,0.07,0.06,0.11,0.08,0.06,0.05,0.06,0.06,0.05,0.08,0.04,0.11,0.06] Value: -0.27
MCTS prediction took 0.839 sec
Probs: [0.2 ,0.02,0.  ,0.02,0.02,0.07,0.09,0.06,0.04,0.02,0.09,0.01,0.02,0.01,0.3 ,0.02]
MCTS only prediction took 0.133 sec
Probs: [0.06,0.06,0.  ,0.06,0.06,0.06,0.06,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07]


In [20]:
config.numMCTSSims = 100
play = [2,1,2,1,2,2]
nn_pred(play)
mcts_pred(play)
mcts_only_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - X - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.013 sec
Probs: [0.06,0.06,0.  ,0.07,0.06,0.11,0.08,0.06,0.05,0.06,0.06,0.05,0.08,0.04,0.11,0.06] Value: -0.27
MCTS prediction took 1.953 sec
Probs: [0.22,0.02,0.  ,0.02,0.02,0.07,0.09,0.06,0.04,0.02,0.09,0.01,0.02,0.01,0.29,0.02]
MCTS only prediction took 0.111 sec
Probs: [0.06,0.06,0.  ,0.06,0.06,0.06,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07]


In [21]:
[x.visit_count for x in r1.children.values()]

[1, 15, 100, 1, 5, 1, 1, 1, 3, 4, 11, 3, 1, 5, 1, 6]